# Mengengerüßt

In [8]:
# Das hier soll im Backend ausgeführt und im Frontend angezeigt werden
from IPython.core.display import display, HTML
from mengengeruest import read_excel_file, select_data, extract_information_through_positionnumbers
from mengengeruest import create_cost_estimate_intern, create_cost_estimate_extern
path = "Data\Angebot.xls"

data = read_excel_file(file_name=path)
select = select_data(data)
convert = extract_information_through_positionnumbers(select)
df_intern = create_cost_estimate_intern(convert)

df_intern["Bezeichnung"] = df_intern["Bezeichnung"].str.replace(r"\n", "<br>", regex=True)
display(HTML(df_intern.to_html(escape=False)))

C:\Users\sandra.nuissl\AppData\Local\Temp\ipykernel_19156\2764338888.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,Pos:,Hersteller,Bezeichnung,Artikelnummer,Menge,Einzelpreis / EUR netto,Gesamtpreis / EUR netto
0,1,None,"Migration SE Tisch mit SchiebeplatteTisch Lift Elektrisch H650-1250 T-Fuß,Typ 10/geradeOption: Größe: StandardBreite Standard: 1800Tiefe Standard: 800Überstand: ohne ÜberstandTyp Platte: 25mm Melamin StdFarbe Tischplatte: Snow WYFarbe Plattenkante: Schwarz 340/N0Untere Säule und Fußausleger: Pearl Schwarz PBInnere Säule(n): Pearl Schwarz PBSchiebeplatte: Facility-Manager SP1Netzkabel für Lifttisch: mit Kabel 3m DE/FR/CHBedienpaneel Lifttisch: Auf/Ab Bedienpaneel SC1Elektrifizierung: NeinPlattenausfräsung: ohneKabelkanal: Standard K00",MSE1T210S,2,"658,89","1.317,78"
1,1.1,None,Migration SE Kabelmanagement TischVertikal Kabelkanal für Lift Elektro650-1250mmGestellfarbe: Amsel MGTisch mit Unterschrank und Kabinett links,MSEA9000,2,"41,50","83,00"
2,2.1,None,"Migration SE Tisch ohne SchiebeplatteTisch Lift Elektrisch H650-1250 T-Fuß,Typ 10/geradeOption: Größe: StandardBreite Standard: 1800Tiefe Standard: 800Überstand: ohne ÜberstandVerkettung: Links LSIVerkettung: NeinTyp Platte: 25mm Melamin StdFarbe Tischplatte: Snow WYFarbe Plattenkante: Schwarz 340/N0Untere Säule und Fußausleger: Pearl Schwarz PBInnere Säule(n): Pearl Schwarz PBNetzkabel für Lifttisch: mit Kabel 3m DE/FR/CHBedienpaneel Lifttisch: Auf/Ab Bedienpaneel SC1Elektrifizierung: NeinKabeldose: ohnePlattenausfräsung: ohneKabelkanal: Standard K00Anbauten Links: NeinAnbauten rechts: NeinBitte ausführen mit Schiebeplatte Artikelnummer: MSE1T210S",MSE1T210,4,"658,89","2.635,56"
3,2.1.1,None,Migration SE Kabelmanagement TischVertikal Kabelkanal für Lift Elektro650-1250mmGestellfarbe: Amsel MG,MSEA9000,4,"41,50","166,00"
4,2.2,None,"Netline Nak Netzanfangskabel D / A / F / BE / CZ schwarz, 1,5 mm²Schuko/BE-Winkelstecker - Buchse GST18/3, flache AusführungKabellänge in mm: ca.: 3000, inkl. Kabelbinder--------------------------------------------Artikelnummer: 6K23007003",AHM,4,"8,07","32,28"
5,2.3,None,"Netbox PB Gehäuse für 5 EinsätzeStandard Typ M, Endkappe abgerundet, Befestigung im Kabelkanal o.ä.,die Module und auch das Gehäuse der Profilbox sind schwarz.Im Lieferumfang: Befestigungslaschen, zum Einstecken in d. StirnkappenVerdrahtung (in mm²): 1,5Stromversorgung: GST18/3-Steckerteil, ohne KabelStromweiterleitung: GST18/3-Buchsenteil, ohne Kabel----------------------------------------5x Schuko (diagonal)----------------------------------------Artikelnummer: 2P15G03013",AHM,4,"28,35","113,40"
6,2.4,None,Divisio Acoustic Screen Auf TischTisch B 1800mmTischadapter: Klemme 25mm PlattenstärkeFarbe Tischadapter: Platin Metallic (Silver) SLOption: Größe: StandardHöhe vom Boden (mm): 1165Screen: SScreen Höhe (mm): 407Screen Breite: 1800Stoff: Atlantic ATAtlantic: Y 67061 AT30Thread color: 416Kante: ZipperReißverschlussfarbe: Silver,90AA1800,4,"319,68","1.278,72"
7,2.5,None,"DIS STS16,1OH,BU, T-Fuss B:1600 T:434",W8S1B1710T,4,"331,01","1.324,04"
8,2.5.1,None,"MTU Hutch,2OH,TU, T-Fuss B:1600 T:209Breite: 1600Tiefe: 209Höhe: 750Typ Kabinett Stoff: AcousticKabinett Stoff: Atlantic ATKabinett Stofffarbe: Y 64089 AT01Stoff Kissen: Atlantic ATStofffarbe Kissen: Y 64089 AT01Plattenfarbe: Snow WYPlattenkante: Gleiche Farbe wie PlatteKorpusfarbe: Snow WYIntegrierte Rückwand: Gleiche Farbe wie KorpusAdaptive Rückwand: NeinAmbiente Licht: NeinStromanschlüsse: Nein powerKabinett Stoffarbe: AT06 Stoffarbe Kissen: AT06nanTisch mit Unterschrank und Kabinett rechts",W8H2T070AT,4,"764,39","3.057,56"
9,3.1,None,"Migration SE Tisch ohne SchiebeplatteTisch Lift Elektrisch H650-1250 T-Fuß,Typ 10/geradeOption: Größe: StandardBreite Standard: 1800Tiefe Standard: 800Überstand: ohne ÜberstandVerkettung: NeinVerkettung: Rechts RSITyp Platte: 25mm Melamin StdFarbe Tischplatte: Snow WYFarbe Plattenkante: Schwarz 340/N0Untere Säule und Fußausleger: Pearl Schwarz PBInnere Säule(n): Pearl Schwarz PBNetzkabel für Lifttisch: mit Kabel 3m DE/FR/CHBedienpaneel Lifttisch: Auf/Ab Bedienpaneel SC1Elektrifizierung

# Technischer Vorbeschrieb

In [1]:
import json
import ollama
ollama.pull(model='llama3.1')

from technischer_Vorbeschrieb import process_pdf, get_criteria

In [2]:
# set variable
pdf_path = "Data\Produktdatenblatt Migration SE.pdf"
furniture = "Bürostuhl"

# execute functions
data = process_pdf(pdf_path)
criteria = get_criteria(furniture=furniture)

In [4]:
criteria

['Abmessungen',
 'Gewicht',
 'Mechanik',
 'Sitzfläche und -höhe',
 'Rückenlehne und -höhe',
 'Armlänge und -form',
 'Füße und Rollgelenke',
 'Belastbarkeit und Lastkapazität',
 'Dynamik und Stabilität',
 'Ergonomie und Anpassungsfähigkeit',
 'Haltbarkeit und Abriebfestigkeit',
 'Nachhaltigkeit und Recyclingfähigkeit',
 'Luftundurchlässigkeit und Brandbeständigkeit',
 'Farbgebung und Oberflächenschutz',
 'Produktionsprozesse und -standards',
 'Zertifizierungen und Normen',
 'Montage- und Demontagesicherheit']

Zusammenfassung

In [5]:
# Pfad zur JSON-Datei
example_json_path = './Data/example.json'

# Beispielzusammenfassungen in einem strukturierten Format (Json file "examples" laden)
with open(example_json_path, 'r', encoding='utf-8') as file:
    examples = json.load(file)

text_to_summarize = data["context"]

In [6]:
modelfile_summary = f'''
FROM llama3.1
SYSTEM deine Aufgabe ist, es Texte für den technischen Vorbeschrieb von Ausschreibungen für Möbelstücken anhand des gegebenen PDF Dokuments zu erstellen.\
Hierbei handelt es sich nicht um eine Bewerbung des Produktes, sondern um eine neutrale und fachliche Aufführung der technischen Voraussetzungen.\
Ähnlich wie in diesen Beispieltexten:\
{examples}\
Daher sollen keine Firmennamen, herrstellerspezifische Produktnamen oder -beschreibungen aufgeführt werden.\
Hinweise:\
- Keine numerischen Aufzählungen\
- Keine Anführungszeichen, wie "" oder ''\
- keine Doppelpunkte, Klammern oder Spiegelstriche\
- Keine Absätze\
- keine formatierungen (wie dick gedruckte Schrift)\
- Keine einleitenden Worte wie "Hier sind die extrahierten Inhalte der Textabschnitte zum Thema ..."\
- Keine abschließenden Worte wie "zu beachten ist ..." oder "Diese Informationen enthalten technische Spezifikationen ..."\
Antwort-Format: Gib ein Dictionary aus, dessen Keys die Schlagworte sind und die Values eine Liste mit den Stichpunkten aus dem PDF Dokument.
'''

ollama.create(model='summary', modelfile=modelfile_summary)

{'status': 'success'}

In [7]:
point = criteria[0]

In [8]:
messages = f'''
Der bereitgestellte Text ist {text_to_summarize}.\ 
Extrahiere aus dem bereitgestellten Text alle Inhalte der Textabschnitte, die sich auf das Thema {point} bezieht.\ 
Stelle sicher, dass alle relevanten Beschreibungen, technischen Spezifikationen und Materialien enthalten sind.\ 
Lasse irrelevante Abschnitte aus. Konzentriere dich auf Informationen, die direkt oder indirekt mit {point} in Zusammenhang stehen,\
und stelle diese technisch präzise dar'''

summary = ollama.generate(
    model='summary', 
    prompt=messages
)
summary["response"]

'Hier sind die extrahierten Informationen zu den Abmessungen der Migration SE-Tische:\n\n**Migration SE höhenverstellbarer Besprechungstisch**\n\n* Höhe: 720-1050 mm\n* Breite:\n\t+ T-Fußgestell: 2000, 2200, 2400 mm\n\t+ 4-Fußgestell: 2800, 3200 mm\n* Tiefe:\n\t+ T-Fußgestell: 1000 mm\n\t+ 4-Fußgestell: 1200, 1400 mm\n\n**Migration SE Bench in H-Form**\n\n* Höhe:\n\t+ Basis: 720-1200 mm\n\t+ Erweitert: 650-1250 mm\n* Breite:\n\t+ 1200-2000 mm\n* Tiefe:\n\t+ 1600, 1700, 1900 mm\n\n**Migration SE Starr**\n\n* Abmessungen:\n\t+ Höhe: keine Angaben\n\t+ Breite: keine Angaben\n\t+ Tiefe: keine Angaben\n\n**Allgemeine Abmessungen (Basis- und Erweiterter Tisch)**\n\n* Höhe:\n\t+ Basis: 740 mm\n\t+ Erweitert: 650-1250 mm\n* Breite:\n\t+ 1200-2000 mm\n* Tiefe:\n\t+ 750, 800, 900 mm'

In [9]:
sum = summary['response'].split("\n")[2:]
sum

['**Migration SE höhenverstellbarer Besprechungstisch**',
 '',
 '* Höhe: 720-1050 mm',
 '* Breite:',
 '\t+ T-Fußgestell: 2000, 2200, 2400 mm',
 '\t+ 4-Fußgestell: 2800, 3200 mm',
 '* Tiefe:',
 '\t+ T-Fußgestell: 1000 mm',
 '\t+ 4-Fußgestell: 1200, 1400 mm',
 '',
 '**Migration SE Bench in H-Form**',
 '',
 '* Höhe:',
 '\t+ Basis: 720-1200 mm',
 '\t+ Erweitert: 650-1250 mm',
 '* Breite:',
 '\t+ 1200-2000 mm',
 '* Tiefe:',
 '\t+ 1600, 1700, 1900 mm',
 '',
 '**Migration SE Starr**',
 '',
 '* Abmessungen:',
 '\t+ Höhe: keine Angaben',
 '\t+ Breite: keine Angaben',
 '\t+ Tiefe: keine Angaben',
 '',
 '**Allgemeine Abmessungen (Basis- und Erweiterter Tisch)**',
 '',
 '* Höhe:',
 '\t+ Basis: 740 mm',
 '\t+ Erweitert: 650-1250 mm',
 '* Breite:',
 '\t+ 1200-2000 mm',
 '* Tiefe:',
 '\t+ 750, 800, 900 mm']